# 🎥 AI CCTV - Complete Backend

### Features:
- ✅ Multi-Model Selection (3 sugar bag models)
- ✅ Video Compression (FFmpeg)
- ✅ Face Detection & Recognition
- ✅ Person Tracker with Timeline
- ✅ Label Scanner (OCR)

### Setup:
1. Upload models to `/MyDrive/ai_cctv/`
2. Runtime → GPU (T4)
3. Run Cell 1, then Cell 2

In [ ]:
#@title 📦 Cell 1: Complete Setup

from google.colab import drive
drive.mount('/content/drive')
!mkdir -p /content/drive/MyDrive/ai_cctv
!mkdir -p /content/faces
!mkdir -p /content/uploads

print('📦 Installing dependencies...')
!pip install flask flask-cors ultralytics opencv-python-headless pyngrok PyJWT easyocr -q
!pip install face_recognition -q
!apt-get install ffmpeg -y -q

import os
import base64

print('📖 Loading EasyOCR...')
import easyocr
ocr_reader = easyocr.Reader(['en'], gpu=True)
print('✅ OCR ready!')

print('👤 Loading face_recognition...')
import face_recognition
print('✅ Face recognition ready!')

APP_CODE_B64 = '...'

with open('app.py', 'wb') as f:
    f.write(base64.b64decode(APP_CODE_B64))

print('\n✅ SETUP COMPLETE!')
print('➡️ Run Cell 2 to start server')

In [ ]:
#@title 🚀 Cell 2: Start Server

from pyngrok import ngrok
import subprocess, threading, time

NGROK_TOKEN = "389KmdejLxjFkXlEBb1RjlZpL1e_6DGkM8mARYQi6qHcHYz6x"  #@param {type:"string"}
if NGROK_TOKEN: ngrok.set_auth_token(NGROK_TOKEN)
ngrok.kill()

def run_flask():
    subprocess.run(['python', 'app.py'])

threading.Thread(target=run_flask, daemon=True).start()
time.sleep(15)

url = ngrok.connect(5000)
print("="*60)
print("🎉 AI CCTV BACKEND IS LIVE!")
print("="*60)
print(f"\n🌐 PUBLIC URL: {url}")
print("\n✅ Features: Detection, Compression, Face Recognition, Person Tracker")
print(f"🔐 Login: demo@aicctv.com / demo123")
print("\n" + "="*60)

while True:
    time.sleep(60)

In [ ]:
#@title 💾 Cell 3: Save Database
import shutil
shutil.copy('/content/aicctv.db', '/content/drive/MyDrive/ai_cctv/aicctv.db')
print('✅ Database saved!')